In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

import numpy as np
import time

In [ ]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TERMINATE_WALLET]

In [ ]:
vega = VegaServiceNull(run_with_console=True, use_full_vega_wallet=False)
vega.start()

#### Create the wallets

In [ ]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

#### Create the settlment asset and mint

In [ ]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

In [ ]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [ ]:
vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

#### Create the market and set up liquidity

In [ ]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=3,
    )

In [ ]:
market_id = vega.all_markets()[0].id
print(market_id)

In [ ]:
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=600,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_MID",
    reference_sell="PEGGED_REFERENCE_MID",
    delta_buy=1,
    delta_sell=1,
    is_amendment=False,
)

vega.forward('1s')
vega.forward('1s')

In [ ]:
# Submit orders which will stay on the book and create best bid / ask 
vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=96,
    wait=False,
)

vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=104,
    wait=False,
)



In [ ]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

#### Do some trading

In [ ]:
# Do a trade which will cross
price = 99
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=False,
)
vega.forward('10s')

# We will now have mark price of 100 


In [ ]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

In [ ]:
vega.submit_liquidity(
        MM_WALLET.name,
        market_id=market_id,
        commitment_amount=2000,
        fee=0.005,
        buy_specs=[
            ("PEGGED_REFERENCE_MID", 1, 1), 
            ("PEGGED_REFERENCE_BEST_BID", 0.4, 1)],
        sell_specs=[
            ("PEGGED_REFERENCE_MID", 1, 1),
            ("PEGGED_REFERENCE_BEST_ASK", 0.4, 1),
        ],
        is_amendment=False,
    )
vega.wait_for_datanode_sync()

In [ ]:
vega.forward('10s')
vega.wait_for_datanode_sync()


liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print(liq_fees)

In [ ]:
num_levels = 11
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print(ask_prices)
print(ask_volumes)


In [ ]:
vega.settle_market(settlement_wallet = TERMINATE_WALLET.name, settlement_price = 95, market_id=market_id)

In [ ]:
vega.forward('1s')

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

    # positions = vega.positions_by_market(wallet_name=wallet.name, market_id=market_id)
    # print(positions)


In [ ]:
vega.stop()